# Figure 3
## Sample-by-Sample Evaluation 

In [15]:
import os

import cv2
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

import analysis.utils as u
from analysis._article_results._helpers import *
import analysis.statistics.sample_metrics as sm

# pio.renderers.default = "browser"

### Load Data

In [16]:
global_measures = sm.load_global_metrics(DATASET_NAME, PROCESSED_DATA_DIR, stimulus_type=STIMULUS_TYPE, metric=None, iteration=1)
global_measures.drop(index=[peyes.constants.ACCURACY_STR, peyes.constants.BALANCED_ACCURACY_STR], inplace=True)    # Drop Acc+Balanced-Acc metrics

fixation_sdt = sm.load_sdt(DATASET_NAME, PROCESSED_DATA_DIR, label=1, stimulus_type=STIMULUS_TYPE, metric=None, iteration=1)
fixation_sdt = fixation_sdt.loc[[peyes.constants.D_PRIME_STR, peyes.constants.F1_STR]]      # Keep only d' and f1 metrics
fixation_sdt = fixation_sdt.rename(index=lambda idx: f"fixation_{idx}")     # Rename index

saccade_sdt = sm.load_sdt(DATASET_NAME, PROCESSED_DATA_DIR, label=2, stimulus_type=STIMULUS_TYPE, metric=None)
saccade_sdt = saccade_sdt.loc[[peyes.constants.D_PRIME_STR, peyes.constants.F1_STR]]        # Keep only d' and f1 metrics
saccade_sdt = saccade_sdt.rename(index=lambda idx: f"saccade_{idx}")        # Rename index

fixation_saccade_sdt = pd.concat([fixation_sdt, saccade_sdt], axis=0)

### (1) Global Metrics
Evaluation metrics for all labels together:
- Cohen's $\Kappa$
- MCC
- $1-NLD$ 

In [17]:
global_stats, global_pvalues, global_nemenyi, sm_global_Ns = sm.friedman_nemenyi(global_measures, [GT1, GT2])

global_pvalues <= ALPHA

gt,MN,RA
metric,,
cohen's_kappa,True,True
complement_nld,True,True
mcc,True,True


In [18]:
pd.concat([global_stats, global_pvalues], axis=1, keys=['Fr', 'p']).stack(1, future_stack=True)

Q             p
metric         gt                         
cohen's_kappa  MN  69.569231  5.011125e-13
               RA  95.596413  2.077126e-18
complement_nld MN  60.562259  3.459800e-11
               RA  95.929919  1.770128e-18
mcc            MN  70.184615  3.747460e-13
               RA  99.384753  3.372091e-19

#### Post-Hoc Analysis

In [19]:
def generate_posthoc_table(post_hoc_results: pd.DataFrame, metric: str) -> (pd.DataFrame, pd.DataFrame):
    post_hoc = pd.concat([
        post_hoc_results.loc[metric, "RA"],
        post_hoc_results.loc[metric, "MN"]
    ], keys=["RA", "MN"]).reorder_levels([1, 0])
    post_hoc.sort_index(level=0, key=lambda idx: idx.map(lambda det: list(post_hoc.columns).index(det)), inplace=True)
    post_hoc.index.names = [u.PRED_STR, u.GT_STR]
    post_hoc.columns.names = [u.PRED_STR]
    
    post_hoc_table = np.full_like(post_hoc, "n.s.", dtype=np.dtypes.StringDType())
    post_hoc_table[post_hoc <= ALPHA] = '*'
    post_hoc_table[post_hoc <= ALPHA / 5] = '**'
    post_hoc_table[post_hoc <= ALPHA / 50] = '***'
    post_hoc_table = pd.DataFrame(post_hoc_table, index=post_hoc.index, columns=post_hoc.columns)
    
    for c, det1 in enumerate(post_hoc.columns):
        for r, det2 in enumerate(post_hoc.index.get_level_values(0).unique()):
            if r >= c:
                post_hoc_table.loc[det2, det1] = ""
            else:
                post_hoc_table.loc[(det1, "MN"), det2] = f"{post_hoc.loc[(det1, "MN"), det2] :.4f}"
                post_hoc_table.loc[(det1, "RA"), det2] = f"{post_hoc.loc[(det1, "RA"), det2] :.4f}"
    return post_hoc_table

**Cohen's Kappa**

In [20]:
post_hoc_kappa = generate_posthoc_table(global_nemenyi, peyes.constants.COHENS_KAPPA_STR)
post_hoc_kappa

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN            n.s.       *    n.s.    n.s.    n.s.      ***
         RA            n.s.      **       *    n.s.    n.s.      ***
ivvt     MN  1.0000               *    n.s.    n.s.    n.s.      ***
         RA  1.0000              **       *    n.s.    n.s.      ***
idt      MN  0.0123  0.0210            n.s.      **    n.s.     n.s.
         RA  0.0060  0.0049            n.s.     ***    n.s.     n.s.
idvt     MN  0.0818  0.1208  0.9982               *    n.s.     n.s.
         RA  0.0479  0.0410  0.9981              **    n.s.     n.s.
engbert  MN  0.9980  0.9932  0.0010  0.0118            n.s.      ***
         RA  0.9920  0.9944  0.0002  0.0029            n.s.      ***
nh       MN  0.9863  0.9951  0.1591  0.4734  0.8274               **
         RA  0.9906  0.9869  0.0854  0.3231  0.7685              ***
remodnav MN  0.0000  0.0000  0.8408  0.4847  0.0000  0.0012         
         RA  0.0000  0.0000  0.7694  0.3912  0.0000  0.0002

**Matthew's Correlation Coefficient (MCC)**

In [21]:
post_hoc_mcc = generate_posthoc_table(global_nemenyi, peyes.constants.MCC_STR)
post_hoc_mcc

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN            n.s.      **    n.s.    n.s.    n.s.      ***
         RA            n.s.      **    n.s.    n.s.    n.s.      ***
ivvt     MN  0.9999               *    n.s.    n.s.    n.s.      ***
         RA  1.0000              **    n.s.    n.s.    n.s.      ***
idt      MN  0.0067  0.0236            n.s.     ***    n.s.     n.s.
         RA  0.0039  0.0094            n.s.     ***    n.s.     n.s.
idvt     MN  0.0831  0.1931  0.9929               *    n.s.     n.s.
         RA  0.0596  0.1104  0.9918              **    n.s.     n.s.
engbert  MN  0.9991  0.9853  0.0007  0.0162            n.s.      ***
         RA  0.9966  0.9833  0.0002  0.0061            n.s.      ***
nh       MN  0.9598  0.9949  0.1744  0.6100  0.7650               **
         RA  0.9195  0.9699  0.1858  0.6414  0.5742               **
remodnav MN  0.0000  0.0001  0.9140  0.4998  0.0000  0.0032         
         RA  0.0000  0.0000  0.8658  0.4016  0.0000  0.0020

**Complement Normalized Levenshtein Distance (1-NLD)**

In [22]:
post_hoc_nld = generate_posthoc_table(global_nemenyi, peyes.constants.COMPLEMENT_NLD_STR)
post_hoc_nld

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN            n.s.    n.s.    n.s.    n.s.    n.s.      ***
         RA            n.s.    n.s.    n.s.    n.s.    n.s.      ***
ivvt     MN  0.9989            n.s.    n.s.    n.s.    n.s.       **
         RA  0.9995            n.s.    n.s.    n.s.    n.s.      ***
idt      MN  0.8947  0.9930            n.s.    n.s.    n.s.        *
         RA  0.6207  0.8787            n.s.    n.s.    n.s.       **
idvt     MN  0.9377  0.9978  1.0000            n.s.    n.s.        *
         RA  0.6809  0.9121  1.0000            n.s.    n.s.       **
engbert  MN  1.0000  0.9951  0.8260  0.8863            n.s.      ***
         RA  1.0000  0.9959  0.4827  0.5451            n.s.      ***
nh       MN  0.9978  1.0000  0.9959  0.9989  0.9919               **
         RA  0.9815  0.9997  0.9812  0.9894  0.9478              ***
remodnav MN  0.0001  0.0017  0.0313  0.0196  0.0001  0.0024         
         RA  0.0000  0.0000  0.0058  0.0039  0.0000  0.0001

#### Global Metrics Figure

In [23]:
global_metrics_fig = sm.global_metrics_distributions_figure(
    global_measures,
    GT1, gt2=GT2, only_box=False
)
global_metrics_fig.update_layout(
    title=None,
    width=1200, height=600,
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
)
global_metrics_fig.show()

### (2) Fixation & Saccade Detection
Evaluating detection of fixations and saccades specifically, using:
- Discriminability Index ($d'$)
- f1 score

In [26]:
sdt_statistics, sdt_pvalues, sdt_dunns, sdt_Ns = sm.friedman_nemenyi(fixation_saccade_sdt, [GT1, GT2])

sdt_pvalues <= ALPHA

gt,MN,RA
metric,,
fixation_d_prime,True,True
fixation_f1,True,True
saccade_d_prime,True,True
saccade_f1,True,True


In [27]:
pd.concat([sdt_statistics, sdt_pvalues], axis=1, keys=['Fr', 'p']).stack(1, future_stack=True)

Fr             p
metric           gt                         
fixation_d_prime MN  55.517419  3.644007e-10
                 RA  79.855335  3.827345e-15
fixation_f1      MN  67.749677  1.182213e-12
                 RA  98.343580  5.559357e-19
saccade_d_prime  MN  55.403599  3.842169e-10
                 RA  71.509434  2.003739e-13
saccade_f1       MN  51.455013  2.399829e-09
                 RA  87.121294  1.199342e-16

In [ ]:
# TODO: analyze posthoc + visualize